In [122]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from pytorch_forecasting import TimeSeriesDataSet

from pytorch_forecasting.metrics import MASE, SMAPE

import torch

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('./datasets/df_machine_sorted.csv', index_col=0)
df.dropna(inplace=True)
df.head()

,start_date,job_name,task_name,start_time,end_time,machine,runtime,plan_cpu,plan_mem,cap_cpu,cap_mem,cpu_usage,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,cap_cpu_100
104146,1970-01-28 15:32:43+08:00,9d28a91e7301a697bad12557,worker,2359963.0,2360077.0,0018d759f400ef5c0457ab9c,114.0,100.0,24.414062,96,384,35.636364,2.060832,32.796875,2.447969,4.208984,9600
139760,1970-01-29 16:01:19+08:00,74474bd870f2447ac939f9cd,worker,2448079.0,2448183.0,0018d759f400ef5c0457ab9c,104.0,600.0,29.296875,96,384,51.043478,0.780721,1.835938,0.441632,0.993164,9600
139884,1970-01-29 16:05:16+08:00,a75f2d5a26b8fff5724fdd6d,PyTorchWorker,2448316.0,2448477.0,0018d759f400ef5c0457ab9c,161.0,50.0,48.828125,96,384,59.483871,3.403646,37.934570,2.487621,6.723633,9600
169267,1970-01-30 11:54:59+08:00,0be403d9f0299123cb12c498,PyTorchWorker,2519699.0,2530096.0,0018d759f400ef5c0457ab9c,10397.0,300.0,292.968750,96,384,8021.102453,4.084847,37.093750,6.160750,6.541992,9600
174797,1970-01-30 14:50:21+08:00,a23b5d2cf2a5c94a8a798ac7,PyTorchWorker,2530221.0,2530335.0,0018d759f400ef5c0457ab9c,114.0,50.0,48.828125,96,384,81.285714,4.360521,37.631836,3.374715,6.723633,9600


In [4]:
df['start_time'] = df['start_time'].apply(lambda x: int(x / 86400))
df['end_time'] = df['end_time'].apply(lambda x: int(x / 86400))


In [5]:
start_value = df['start_time'].iloc[0]
df['start_time'] = df['start_time'] - start_value
df['start_time'] = df['start_time'].astype('int')
df['end_time'] = df['end_time'].apply(lambda x: int(x - start_value))
df['end_time'] = df['end_time'] + df['runtime']
df['end_time'] = df['end_time'].astype('int')
df.head()

,start_date,job_name,task_name,start_time,end_time,machine,runtime,plan_cpu,plan_mem,cap_cpu,cap_mem,cpu_usage,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,cap_cpu_100
104146,1970-01-28 15:32:43+08:00,9d28a91e7301a697bad12557,worker,0,114,0018d759f400ef5c0457ab9c,114.0,100.0,24.414062,96,384,35.636364,2.060832,32.796875,2.447969,4.208984,9600
139760,1970-01-29 16:01:19+08:00,74474bd870f2447ac939f9cd,worker,1,105,0018d759f400ef5c0457ab9c,104.0,600.0,29.296875,96,384,51.043478,0.780721,1.835938,0.441632,0.993164,9600
139884,1970-01-29 16:05:16+08:00,a75f2d5a26b8fff5724fdd6d,PyTorchWorker,1,162,0018d759f400ef5c0457ab9c,161.0,50.0,48.828125,96,384,59.483871,3.403646,37.934570,2.487621,6.723633,9600
169267,1970-01-30 11:54:59+08:00,0be403d9f0299123cb12c498,PyTorchWorker,2,10399,0018d759f400ef5c0457ab9c,10397.0,300.0,292.968750,96,384,8021.102453,4.084847,37.093750,6.160750,6.541992,9600
174797,1970-01-30 14:50:21+08:00,a23b5d2cf2a5c94a8a798ac7,PyTorchWorker,2,116,0018d759f400ef5c0457ab9c,114.0,50.0,48.828125,96,384,81.285714,4.360521,37.631836,3.374715,6.723633,9600


In [6]:
max_encoder_length: int = 60
max_prediction_length: int = 20

# training_cutoff = df['end_time'].max() - max_prediction_length
# filter = df['start_time'] <= training_cutoff
# df = df.where(filter)
df.head()

,start_date,job_name,task_name,start_time,end_time,machine,runtime,plan_cpu,plan_mem,cap_cpu,cap_mem,cpu_usage,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,cap_cpu_100
104146,1970-01-28 15:32:43+08:00,9d28a91e7301a697bad12557,worker,0,114,0018d759f400ef5c0457ab9c,114.0,100.0,24.414062,96,384,35.636364,2.060832,32.796875,2.447969,4.208984,9600
139760,1970-01-29 16:01:19+08:00,74474bd870f2447ac939f9cd,worker,1,105,0018d759f400ef5c0457ab9c,104.0,600.0,29.296875,96,384,51.043478,0.780721,1.835938,0.441632,0.993164,9600
139884,1970-01-29 16:05:16+08:00,a75f2d5a26b8fff5724fdd6d,PyTorchWorker,1,162,0018d759f400ef5c0457ab9c,161.0,50.0,48.828125,96,384,59.483871,3.403646,37.934570,2.487621,6.723633,9600
169267,1970-01-30 11:54:59+08:00,0be403d9f0299123cb12c498,PyTorchWorker,2,10399,0018d759f400ef5c0457ab9c,10397.0,300.0,292.968750,96,384,8021.102453,4.084847,37.093750,6.160750,6.541992,9600
174797,1970-01-30 14:50:21+08:00,a23b5d2cf2a5c94a8a798ac7,PyTorchWorker,2,116,0018d759f400ef5c0457ab9c,114.0,50.0,48.828125,96,384,81.285714,4.360521,37.631836,3.374715,6.723633,9600


In [9]:
df.start_time.unique()

array([ 0,  1,  2,  3,  6,  7,  8,  9, 11, 12, 13, 14, 16, 18, 19, 20, 21,
       22, 23, 24, 26, 28, 29, 30, 31, 33, 35, 41, 43, 44, 45, 46, -2, -1,
        4,  5, 15, 17, 27, 32, 34, 36, 37, 38, 39, 40, 42, 47, 10, 25, -5,
       -4, -3, -7, -6, -8, -9])

In [123]:
mase = MASE()
mape = SMAPE()

In [128]:
pred = torch.ones((100, 1))
target = torch.randint((100, 1))
target += 3

print(mape.loss(pred, target))

# TimeSeriesDataSet(df, time_idx='start_time', target=['cpu_usage', 'avg_mem'], group_ids=['cap_cpu'])

tensor([[1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1.2000],
        [1